# Positron emission tomography (PET) data realignment example

This example shows how to estimate the head motion of a PET dataset using `NiFreeze`.

The notebook uses the `sub-02` dataset that was generated synthetically from
a real PET dataset by adding random motion. The dataset can be installed from
[GIN G-node](https://gin.g-node.org/nipreps-data/tests-nifreeze):

```
$ datalad install -g https://gin.g-node.org/nipreps-data/tests-nifreeze.git
```

after which the environment variable `TEST_DATA_HOME` will need to be set to
point to the corresponding folder.

In [ ]:
from os import getenv
from pathlib import Path

from nifreeze.data.pet import from_nii

# Install test data from gin.g-node.org:
#   $ datalad install -g https://gin.g-node.org/nipreps-data/tests-nifreeze.git
# and point the environment variable TEST_DATA_HOME to the corresponding folder
DATA_PATH = Path(getenv("TEST_DATA_HOME", str(Path.home() / "nifreeze-tests")))
WORKDIR = Path.home() / "tmp" / "nifreezedev" / "pet_data"
WORKDIR.mkdir(parents=True, exist_ok=True)

OUTPUT_DIR = WORKDIR / "motion_estimation"
OUTPUT_DIR.mkdir(exist_ok=True, parents=True)

pet_file = (
    DATA_PATH / "pet_data" / "sub-02" / "ses-baseline" / "pet" / "sub-02_ses-baseline_pet.nii.gz"
)
json_file = (
    DATA_PATH / "pet_data" / "sub-02" / "ses-baseline" / "pet" / "sub-02_ses-baseline_pet.json"
)

pet_dataset = from_nii(pet_file, temporal_file=json_file)

Let's first preprocess the data: we will smooth and threshold them.

In [ ]:
import nibabel as nb
import numpy as np
from nibabel.processing import smooth_image

smooth_fwhm = 10.0
thresh_pct = 20.0

pet_img = nb.load(pet_file)
pet_dataobj = pet_img.get_fdata()

smoothed_img = smooth_image(nb.Nifti1Image(pet_dataobj, pet_img.affine), smooth_fwhm)
thresh_val = np.percentile(smoothed_img.get_fdata(), thresh_pct)
pet_dataobj[pet_dataobj < thresh_val] = 0

preproc_pet_img_file = WORKDIR / "sub-02_ses-baseline_pet_desc-preproc.nii.gz"

nb.save(nb.Nifti1Image(pet_dataobj, pet_img.affine), preproc_pet_img_file)

We will now create the PET dataset object.

In [ ]:
pet_dataset = from_nii(preproc_pet_img_file, temporal_file=json_file)

pet_dataset

## Model fitting and motion correction

The `nifreeze.model.BSplinePETModel` is a predictive model that employs
a B-Spline-based interpolation method to model temporal signal evolution
across PET frames. We will demonstrate how to estimate a held-out volume
employing this model.

In [ ]:
from nifreeze.model import BSplinePETModel

n_ctrl = 3
order = 3

# Create model with the reduced dataset
model = BSplinePETModel(dataset=pet_dataset, n_ctrl=n_ctrl, order=order)

Let's now ask the model for a prediction at the time point pointed by the
index value `2`. By calling `model.fit_predict` the model is fit on the
remaining frames, and a prediction is requested on the time indicated by
the provided index.

In [ ]:
index = 2
predicted = model.fit_predict(index=index)

We now save the uncorrected and corrected data so that we can visualize the
difference.

In [ ]:
import nibabel as nb

# before
nifti_img_before = nb.Nifti1Image(pet_dataset[index][0], pet_dataset.affine)
output_path_before = "before_mc.nii"
nifti_img_before.to_filename(output_path_before)

# after
nifti_img_after = nb.Nifti1Image(predicted, pet_dataset.affine)
output_path_after = "after_mc.nii"
nifti_img_after.to_filename(output_path_after)

Let's now visualize a number of axial, sagittal and coronal slices of the
uncorrected and corrected data.

In [ ]:
from niworkflows.viz.notebook import display

moving_image = output_path_after
fixed_image = output_path_before
obj = display(
    fixed_image,
    moving_image,
    fixed_label="PET_before",
    moving_label="PET_after",
)

## Motion estimation

We now want to have an estimate of the motion that the model corrects. We will
need to instantiate the `nifreeze.estimator.PETMotionEstimator`, which will
take an instance of the model. We will call `run` to get the parameters of the
affine transform estimation. As the estimator runs, a registration process
estimates the transform parameters between the held-out volume and the
estimated volume: for each held-out volume index, the transform parameters are
saved to the dataset.

In [ ]:
from nifreeze.estimator import Estimator

strategy = "linear"
estimator = Estimator(model=model, strategy=strategy)

# Run the estimator
estimator.run(pet_dataset, omp_nthreads=4)

# Get the estimated motion affines
affines = pet_dataset.motion_affines

Let's now visualize the estimated motion: we will plot the translation and
rotation components in the affine transform for each axis.

In [ ]:
import numpy as np
import pandas as pd

from nifreeze.registration.utils import compute_fd_from_motion, extract_motion_parameters

# Assume `affines` is the list of affine matrices computed earlier
motion_parameters = []

for _idx, affine in enumerate(affines):
    tx, ty, tz, rx, ry, rz = extract_motion_parameters(affine)
    motion_parameters.append([tx, ty, tz, rx, ry, rz])

motion_parameters = np.array(motion_parameters)
estimated_fd = compute_fd_from_motion(motion_parameters)

In [ ]:
# Set up the matplotlib figure
import matplotlib.pyplot as plt

%matplotlib inline

from nifreeze.viz.motion_viz import plot_volumewise_motion

plot_volumewise_motion(np.arange(len(estimated_fd)), motion_parameters)

plt.show()

For the dataset used in this example, we have access to the ground truth motion parameters that were used to corrupt the motion-free dataset. Let's now plot the ground truth motion to enable a visual comparison with the estimated motion.

In [ ]:
from nifreeze.viz.motion_viz import plot_volumewise_motion

%matplotlib inline

motion_gt_fname = (
    DATA_PATH
    / "pet_data"
    / "sub-02"
    / "ses-baseline"
    / "pet"
    / "sub-02_ses-baseline_ground_truth_motion.csv"
)
motion_gt_df = pd.read_csv(motion_gt_fname)

frames = motion_gt_df["frame"].to_numpy()

# Construct motion_params array with shape (n_frames, 6): [trans_x, trans_y, trans_z, rot_x, rot_y, rot_z]
motion_cols = ["trans_x", "trans_y", "trans_z", "rot_x", "rot_y", "rot_z"]
motion_params = motion_gt_df[motion_cols].to_numpy()

plot_volumewise_motion(frames, motion_params)

plt.tight_layout()
plt.show()

Let's plot the estimated and the ground truth framewise displacement.

In [ ]:
from nifreeze.viz.motion_viz import plot_framewise_displacement

fd = pd.DataFrame({"estimated": estimated_fd, "gt": motion_gt_df["framewise_displacement"].values})
plot_framewise_displacement(fd, labels=["Estimated", "Ground truth"])

plt.show()